In [1]:
!pip install torch
!pip install torchvision

  Using cached https://files.pythonhosted.org/packages/62/94/5430ebaa83f91cc7a9f687ff5238e26164a779cca2ef9903232268b0a318/Pillow-5.3.0-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


Давайте напишем какую нить  простую сеть на numpy
# Давайте напишем какую нить совсем простую сетку на numpy

In [4]:
import numpy as np
from torchvision.datasets import mnist
from torch.utils.data import Dataset, DataLoader
import torch
import os
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 1, 784, 10, 10

# Create random input and output data
def toNumpy(x):
    return np.array(x,np.float32).flatten()/255.0


def label(label):
        r=np.zeros(10,np.float32);
        r[label]=1.0;
        return r;
tr=mnist.FashionMNIST(os.path.curdir+"/fmnist",download=True,transform=toNumpy,target_transform=label,train=True)


# Randomly initialize weights
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6

num=0;
for i_batch, sample_batched in enumerate(tr):
    # Forward pass: compute predicted y
    x=np.expand_dims(sample_batched[0],axis=0)
    y=np.expand_dims(sample_batched[1],axis=0)
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)

    # Compute and print loss
    loss = np.square(y_pred - y).sum()
    if num%10000==0:
      print(loss)
    num=num+1
    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)

    # Update weights
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

8104.842525402983
9.890480577252786
1.0
1.0
1.0
1.0


О кей замечательно а зачем нам тогда torch, давайте попробуем написать тоже самое на нем
# Как то очень громоздко, да и ускорителя хочется, давайте попробуем PyTorch

In [5]:
dtype = torch.float32
device = torch.device("cpu")
dataloader=DataLoader(tr,batch_size=4)

w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

for i_batch, sample_batched in enumerate(dataloader):
    # Forward pass: compute predicted y
    x=sample_batched[0]
    y=sample_batched[1]
    # Forward pass: compute predicted y
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if num%1000==0:
      print(loss)
    num=num+1

    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # Update weights using gradient descent
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

15009.474609375
1714.2652587890625
357.0181884765625
39.878719329833984
287.02288818359375
595.791015625
20.413578033447266
48.27613830566406
77.92149353027344
91.75447082519531
7.335393905639648
113.30887603759766
9.752111434936523
80.4443359375
7.729575157165527


Какой то очень похожий код получился, в чем же соль?
# Autograd

In [6]:
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

for i_batch, sample_batched in enumerate(dataloader):
    # Forward pass: compute predicted y
    x=sample_batched[0]
    y=sample_batched[1]
    # Forward pass: compute predicted y
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum()
    if num%1000==0:
      print(loss.item())
    num = num + 1
    loss.backward()

    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # Manually zero the gradients after updating weights
        w1.grad.zero_()
        w2.grad.zero_()

47525.921875
638.318115234375
155.62283325195312
289.56805419921875
23.228336334228516
527.4720458984375
4.0
90.97450256347656
352.0081787109375
4.673168659210205
137.875732421875
20.235248565673828
287.8556213378906
25.872745513916016
34.8391227722168


Уже лучше но может быть в библиотеке есть еще какие нить полезные абстракции?
# Welcome nn.modules

In [7]:
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(reduction='sum')

for i_batch, sample_batched in enumerate(dataloader):
    # Forward pass: compute predicted y
    x=sample_batched[0]
    y=sample_batched[1]
    y_pred = model(x)

    # Compute and print loss
    loss = loss_fn(y_pred, y)
    model.zero_grad()
    if num%1000==0:
      print(loss.item())
    num=num+1
    loss.backward()

    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

4.2322163581848145
5.118720054626465
3.7685601711273193
5.09650182723999
3.7381396293640137
3.3226401805877686
3.92453932762146
4.040871620178223
4.804718017578125
3.307461738586426
4.164682865142822
3.152167558670044
3.3927435874938965
3.3534796237945557
3.310598850250244


А нужно ли самим апдейтить веса?

In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for i_batch, sample_batched in enumerate(dataloader):
    # Forward pass: compute predicted y
    x=sample_batched[0]
    y=sample_batched[1]
    y_pred = model(x)

    # Compute and print loss
    loss = loss_fn(y_pred, y)
    if num%1000==0:
      print(loss.item())
    num=num+1

    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

3.6236021518707275
3.583860158920288
3.476802110671997
4.218475341796875
3.53511905670166
2.871598243713379
3.5844526290893555
3.619049310684204
4.245276927947998
3.0241246223449707
3.4729809761047363
2.886005401611328
3.2664787769317627
2.715991258621216
2.8421385288238525


А если мы хотим свой модуль?
# Как написать кастомный модуль

In [9]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred



# Construct our model by instantiating the class defined above
model = TwoLayerNet(D_in, H, D_out)

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for i_batch, sample_batched in enumerate(dataloader):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if num%1000==0:
      print(loss.item())
    num=num+1   
    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

4.695863246917725
1.2893601655960083
0.7731939554214478
0.43944790959358215
0.23118431866168976
0.11921994388103485
0.06225814297795296
0.03325456380844116
0.018173672258853912
0.010128169320523739
0.00573276961222291
0.0032822296489030123
0.0018452862277626991
0.0010423548519611359
0.0005955630913376808
